# Run Distribution Learning Benchmark

In [1]:
from hybrid_transformer.configs.tasks.distribution_learning import DistributionLearningConfig
from hybrid_transformer.configs.trainers.trainer import TrainerConfig

from hybrid_transformer.utils.datasets.auto import AutoDataset
from hybrid_transformer.utils.tokenizers.auto import AutoTokenizer

from hybrid_transformer.utils.runtime import set_seed, get_device

from hybrid_transformer.configs.models.model import ModelConfig
from hybrid_transformer.models.auto import AutoModel

from hybrid_transformer.configs.trainers.trainer import TrainerConfig
from hybrid_transformer.trainers.trainer import Trainer

%load_ext autoreload
%autoreload 2

/home/adam/miniconda3/envs/hybrid-transformer/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-25 13:45:02.175127: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-25 13:45:02.177046: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-25 13:45:02.201972: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-25 13:45:02.201988: E external/local_xla/xla/stream_executor/cu

In [2]:
# 1. Args and Config 

args = {}
args['out_dir'] = 'guacamol'
args['experiment_config'] = 'guacamol' # GPT or HT

device = get_device()
set_seed()

task_config = DistributionLearningConfig(validate=False, split='test', target_label='zale', out_dir='./results/test/')
model_config = ModelConfig()
trainer_config = TrainerConfig(batch_size=8, eval_iters=2)

dataset = AutoDataset.from_config(task_config)
tokenizer = AutoTokenizer.from_config(task_config)
model = AutoModel.from_config(model_config)
model.train()
trainer = Trainer(config=trainer_config, model=model, train_dataset=dataset, eval_dataset=dataset, tokenizer=tokenizer)

Random seed set to 42
number of parameters: 38.06M
Running on a single device!
tokens per iteration will be: 40,960
Using cuda device
compiling the model... (takes a ~minute)
num decayed parameter tensors: 63, with 38,115,840 parameters
num non-decayed parameter tensors: 25, with 12,800 parameters
using fused AdamW: True


In [15]:


outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=inputs['labels'], target=inputs['target'], eos_mask=inputs['eos_mask'])

In [16]:
outputs.keys()

dict_keys(['unsupervised_loss', 'supervised_loss', 'lm_logits', 'prediction', 'embedding'])

In [8]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'target', 'eos_mask'])

In [17]:
inputs['attention_mask']

In [19]:
inputs['target']

tensor([9.3366e-08, 7.6793e-02], device='cuda:0')

In [20]:
if outputs['supervised_loss'] is not None:
    

tensor(0.2705, device='cuda:0', grad_fn=<MseLossBackward0>)

In [ ]:
# helps estimate an arbitrarily accurate loss over either split using many batches


In [21]:
model.generate()

TypeError: GPT.generate() missing 2 required positional arguments: 'idx' and 'max_new_tokens'

In [31]:
import torch



GPT(
  (transformer): ModuleDict(
    (wte): Embedding(588, 512)
    (wpe): Embedding(128, 512)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x HybridTransformerBlock(
        (ln_1): LayerNorm()
        (attn_1): HybridSelfAttention(
          (q_proj): Linear(in_features=512, out_features=512, bias=False)
          (kv_proj): Linear(in_features=512, out_features=1024, bias=False)
          (out_proj): Linear(in_features=512, out_features=512, bias=False)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (fc): Linear(in_features=512, out_features=2048, bias=False)
          (gelu): GELU(approximate='none')
          (proj): Linear(in_features=2048, out_features=512, bias=False)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=512, out_fea

In [32]:
out

{'train': tensor(1.7767), 'valid': 0.0, 'val': tensor(1.8147)}

In [3]:
trainer.estimate_loss()

{'train': tensor(1.8007), 'valid': 0.0, 'val': tensor(1.8385)}